In [23]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as  mape
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

from torchmetrics import MeanSquaredLogError


In [25]:
data = pd.read_csv("final.csv")

In [26]:
data = data[data["Area"].isnull() == False]
data.shape

(34953, 16)

In [27]:
data["Price"] = data["Price"]/data["Area"]

In [28]:
data = data[['House Direction', 'Balcony Direction', 'Bedrooms', 'Toilets',
       'Legits', 'Floors', 'Facade', 'Entrance', 'X', "Y",
       'Price']]

In [29]:
data = data[data["Price"] > 100]

In [30]:
IQR = data.Price.quantile(0.75) - data.Price.quantile(0.25)
Lower_fence = data.Price.quantile(0.25) - (IQR * 3)
Upper_fence = data.Price.quantile(0.75) + (IQR * 3)

data = data[(data.Price >= Lower_fence) & (data.Price <= Upper_fence)]
data.Price.describe()

count    29570.000000
mean      1316.770876
std       1248.936842
min        100.500000
25%        260.666502
50%       1044.968705
75%       1818.181818
max       6722.689076
Name: Price, dtype: float64

In [31]:
hotdf = pd.get_dummies(data)

In [32]:
hotdf["Entrance"].fillna(value=hotdf["Entrance"].mode().values[0], inplace=True)

In [33]:
features = hotdf.columns.tolist()
features.remove("Price")

In [34]:
train, test = train_test_split(hotdf, test_size = 0.2, random_state = 0)
train, dev = train_test_split(train, test_size = 0.1, random_state = 0)

In [35]:
scaler = MinMaxScaler()

train[features] = scaler.fit_transform(train[features])

test[features] = scaler.transform(test[features])
dev[features] = scaler.transform(dev[features])

In [36]:
dev[features].head()

,Facade,Entrance,X,Y,House Direction_None,House Direction_bắc,House Direction_nam,House Direction_tây,House Direction_tây bắc,House Direction_tây nam,...,Floors_10+,Floors_2,Floors_3,Floors_4,Floors_5,Floors_6,Floors_7,Floors_8,Floors_9,Floors_None
22235,0.009783,0.015228,0.563135,0.558745,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32988,0.004348,0.015228,0.545976,0.087881,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22798,0.003261,0.015228,0.480748,0.613380,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
12924,0.010870,0.015228,0.525457,0.698002,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
30083,0.008696,0.015228,0.479429,0.666881,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [37]:
dev["Price"]

22235    1716.666667
32988     139.240506
22798    1000.000000
12924     207.142857
30083    2307.692308
            ...     
4194      112.830432
18790    1140.000000
18518     272.352941
6214      314.000000
23923    1181.818182
Name: Price, Length: 2366, dtype: float64

In [38]:
linear_re = LinearRegression()


In [39]:
linear_re.fit(train[features], train["Price"])



LinearRegression()

In [40]:
pred = linear_re.predict(dev[features])
print(mape(dev["Price"], pred))

1.0665626910542862


In [41]:
pred = linear_re.predict(test[features])
print(mape(test["Price"], pred))

1.0854577674792925
